# Imports & Globals

In [1]:
import pickle
import pandas as pd
import numpy as np
from yaml import safe_load
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

2022-10-27 13:08:30.971086: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 13:08:31.102001: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-27 13:08:31.536775: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/israel/miniconda3/envs/tf/lib/:/home/israel/miniconda3/envs/tf/lib/:/home/israel/miniconda3/envs/tf/lib/:/home/israel/miniconda3/envs/tf/lib/:/home/israel/miniconda3/envs/tf/lib/:/home/israel/miniconda3/e

In [2]:
from StringProcessing import preprocessInput, encode_pad_tag, balance_complete_set

In [3]:
from SiameseModel import CosineDistance, ContrastiveLoss 

tf.version:  2.10.0
tf.keras.version:  2.10.0
tf.config.devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-10-27 13:08:32.444067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 13:08:32.476419: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 13:08:32.476652: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 13:08:32.476918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
from ModelEvaluation import find_Model_ranks

In [5]:
#Maximum sequence length including padding
global MAXLEN
with open('LSTM65.yaml', 'r') as file:
    model_config = safe_load(file)
    MAXLEN = model_config['MAXLEN']

# Pickled Datasets

In [6]:
dUnique_df = pd.read_pickle("../data/dUnique_df.pkl")

--------------------------------

# Build model, load weights and evaluate on test data

In [7]:
model = tf.keras.models.load_model(
    './saved_models/{}'.format(model_config["Model_Name"]), 
    custom_objects = {
        'CosineDistance': CosineDistance,
        'ContrastiveLoss': ContrastiveLoss
    }, 
    compile=True, 
    options=None
)
model.summary()

2022-10-27 13:08:32.991658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 13:08:32.991865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 13:08:32.991964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 13:08:32.992098: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 13:08:32.992199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

2022-10-27 13:08:39.068559: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-27 13:08:39.077226: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-27 13:08:39.144828: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-27 13:08:39.153262: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-27 13:08:39.163205: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_sh

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 model (Functional)             (None, 65)           115065      ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

---------------------------------------------------------------

# Evaluation

## Unique target labels

In [8]:
dUnique_df.head()

,dUnique_label,dUnique_seq_padded
0,AAI,"[1, 1, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,AARONSROD,"[1, 1, 18, 15, 14, 19, 18, 15, 4, 0, 0, 0, 0, ..."
2,ABIES ALBA,"[1, 2, 9, 5, 19, 27, 1, 12, 2, 1, 0, 0, 0, 0, ..."
3,ABSINTHE,"[1, 2, 19, 9, 14, 20, 8, 5, 0, 0, 0, 0, 0, 0, ..."
4,ABSINTHE WORMWOOD,"[1, 2, 19, 9, 14, 20, 8, 5, 27, 23, 15, 18, 13..."


In [9]:
dUnique_df.index.size

1879

In [10]:
latin_2_common = preprocessInput('../data/vocabulary_LB_to_common_names_09082022.tsv', maxlen= MAXLEN, reflexive=True, balance = False, noise = None, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lb        1299 non-null   object
 1   mappings  1299 non-null   object
dtypes: object(2)
memory usage: 20.4+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['lb', 'mappings'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 321 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------

In [11]:
common_2_latin = preprocessInput('../data/positive-unmapped-pairs-common-name-or-latin-binomial-copies.tsv', maxlen= MAXLEN, reflexive=True, balance = False, noise = None, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   common_name     728 non-null    object
 1   latin_binomial  728 non-null    object
dtypes: object(2)
memory usage: 11.5+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['common_name', 'latin_binomial'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 728 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 2 columns):
 #   Column  Non-Null Count 

In [12]:
gsrs_2_common = preprocessInput('../data/positive-unmapped-pairs-gsrs-name-to-common-name.tsv', maxlen= MAXLEN, reflexive=True, balance = False, noise = None, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10838 entries, 0 to 10837
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         10838 non-null  object
 1   common_name  199 non-null    object
dtypes: object(2)
memory usage: 169.5+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['name', 'common_name'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 10639 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 0 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  --

In [13]:
# vocab = pd.read_csv('../data/lb_to_common_names.csv')
# vocab = vocab[['latin_binomial_clean','common_name_clean']].rename(columns={'latin_binomial_clean': "x", 'common_name_clean': "y"})
# vocab.info()

In [14]:
equivalence_set = pd.concat([latin_2_common, common_2_latin, gsrs_2_common], ignore_index=True)
equivalence_set.drop_duplicates(subset=['x', 'y'], inplace=True)

In [15]:
equivalence_set[equivalence_set["x"] == "CRANBERRY"]

,x,y
2230,CRANBERRY,VACCINIUM MACROCARPON


## True Positives

In [16]:
matches = encode_pad_tag(preprocessInput('../data/NP_FAERS_mapped_20220215.csv', maxlen=MAXLEN, reflexive=False, noise=None, balance=False), match=1, distance=0, maxlen=MAXLEN)
matches.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5358 entries, 0 to 5357
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   FAERS_drug_match  5358 non-null   object
 1   lookup_value      5358 non-null   object
dtypes: object(2)
memory usage: 83.8+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['FAERS_drug_match', 'lookup_value'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 256 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 487 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4615 entries, 0 to 4614
Data columns (total 2 columns):
 #   Column

,x,y,Processed_x,Processed_y,Match,Distance
0,ASHWAGANDHA,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 0, 0, 0, ...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
1,ASHWAGANDHA CATS CLAW PAU DARCO RHODIOLA STEMM...,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 3, 1,...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
2,ASHWAGANDHA EXTRACT,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 5, 24...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
3,ASHWAGANDHA ROOT EXTRACT,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 18, 1...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
4,ASHWAGANDHA WITHANIA SOMNIFERA,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 23, 9...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0


### ASHWAGANDA

In [17]:
matches.loc[1, "x"]

'ASHWAGANDHA CATS CLAW PAU DARCO RHODIOLA STEMMACANTHA'

In [33]:
predicts = model.predict([np.tile(matches.loc[1, "Processed_x"], (dUnique_df.index.size,1)), np.stack(dUnique_df['dUnique_seq_padded'])]).astype('float32')
top5 = predicts.flatten().argsort()

59/59 [==============================] - 0s 4ms/step


In [34]:
# Top-5 smalles distances
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'ASHWAGANDHA',
 'rank2': 'ASVAGANDHA',
 'rank3': 'ASHWAGANDA',
 'rank4': 'WITHANIA SOMNIFERA',
 'rank5': 'GANODERMA LUCIDUM'}

### Echinacea

In [35]:
Echinacea = matches[matches["y"].str.contains("ECHINACEA")].head(1)
Echinacea

,x,y,Processed_x,Processed_y,Match,Distance
949,ASCORBIC ACID ECHINACEA PURPUREA ZINC CITRATE ...,ECHINACEA,"[1, 19, 3, 15, 18, 2, 9, 3, 27, 1, 3, 9, 4, 27...","[5, 3, 8, 9, 14, 1, 3, 5, 1, 0, 0, 0, 0, 0, 0,...",1,0


In [39]:
i = Echinacea.index.values[0]
matches["x"][i]
# matches.loc[i, "x"]

'ASCORBIC ACID ECHINACEA PURPUREA ZINC CITRATE ZINC GLUCONATE'

In [41]:
predicts = model.predict([np.tile(matches.loc[i, "Processed_x"], (dUnique_df.index.size,1)), np.stack(dUnique_df['dUnique_seq_padded'])]).astype('float32')
top5 = predicts.flatten().argsort()

59/59 [==============================] - 0s 4ms/step


In [42]:
# Top-5 smalles distances
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'ASIATIC CORNELIAN CHERRY',
 'rank2': 'CHUI XU SHANG LU',
 'rank3': 'SICHUAN PAGODATREE',
 'rank4': 'ASIATIC DOGWOOD',
 'rank5': 'SUAN CHENG'}

### Cranberry

In [43]:
cranberry = dUnique_df[dUnique_df["dUnique_label"].str.contains("CRANBERRY")]
cranberry

,dUnique_label,dUnique_seq_padded
510,CRANBERRY,"[3, 18, 1, 14, 2, 5, 18, 18, 25, 0, 0, 0, 0, 0..."


In [44]:
i = cranberry.index
dUnique_df.loc[i, "dUnique_label"]

510    CRANBERRY
Name: dUnique_label, dtype: object

In [45]:
predicts = model.predict([
    np.tile(dUnique_df.loc[i, "dUnique_seq_padded"].to_list(), (dUnique_df.index.size, 1)),
    np.stack(dUnique_df['dUnique_seq_padded'])
]).astype('float32')

top5 = predicts.flatten().argsort()

59/59 [==============================] - 0s 2ms/step


In [46]:
# Top-5 smalles distances
m = max(top5)
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'CRANBERRY',
 'rank2': 'VACCINIUM MACROCARPON',
 'rank3': 'CANDLEBERRY',
 'rank4': 'CHINABERRY',
 'rank5': 'EUROPEAN BARBERRY'}

## True  Negatives

In [28]:
find_Model_ranks(model, df=matches.sample(20), targets= dUnique_df, equivalence_set=equivalence_set, ranks=5, find_related_rank=False, report_distances=False)

,x,y,Processed_x,Processed_y,Match,Distance,rank1,rank2,rank3,rank4,rank5
2012,CRANBERRY WITH VITAMIN D,CRANBERRY,"[3, 18, 1, 14, 2, 5, 18, 18, 25, 27, 23, 9, 20...","[3, 18, 1, 14, 2, 5, 18, 18, 25, 0, 0, 0, 0, 0...",1,0,EPIMEDIUM GRANDIF,RUNBYTHEGROUND,CRANBERRY,BARRENWORT,CANDLEBERRY
3517,ASHUAGANDA,WITHANIA SOMNIFERA,"[1, 19, 8, 21, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...","[23, 9, 20, 8, 1, 14, 9, 1, 27, 19, 15, 13, 14...",1,0,ASHWAGANDHA,ASHWAGANDA,ASVAGANDHA,ASGANDH,WITHANIA SOMNIFERA
908,GYMNEMA SYLVESTREPECTIN,MIRACLEFRUIT,"[7, 25, 13, 14, 5, 13, 1, 27, 19, 25, 12, 22, ...","[13, 9, 18, 1, 3, 12, 5, 6, 18, 21, 9, 20, 0, ...",1,0,GYMNEMA SYLVESTRE,MIRACLEFRUIT,GYMNEMA,ROTTEN CHEESEFRUIT,GUANG GUO GAN CAO
2247,HORSERADISH GARLIC VITAMIN C,GARLIC,"[8, 15, 18, 19, 5, 18, 1, 4, 9, 19, 8, 27, 7, ...","[7, 1, 18, 12, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1,0,HEALINGHERB,CHRYSOJASMINUM FRUTICANS,CHINESE LICORICE,TEPHROSIA VIRGINIANA,CHINESE LICORICE GLYCYRRHIZA URALENSIS HOC
988,NOOTROBOX RISE BACOPA MONNIERI RHODIOLA ROSEA ...,RHODIOLA,"[14, 15, 15, 20, 18, 15, 2, 15, 24, 27, 18, 9,...","[18, 8, 15, 4, 9, 15, 12, 1, 0, 0, 0, 0, 0, 0,...",1,0,LONICERA HYPOGLAUCA,TOOTHED BURCLOVER,MEDICAGO POLYMORPHA,RHODIOLA ROSEA,TOOTHED MEDIC
2757,ASIAN GINSENG GINGER SICHUAN PEPPER,ZINGIBER OFFICINALE,"[1, 19, 9, 1, 14, 27, 7, 9, 14, 19, 5, 14, 7, ...","[26, 9, 14, 7, 9, 2, 5, 18, 27, 15, 6, 6, 9, 3...",1,0,ASIAN GINSENG,ASHWAGANDHA,ASVAGANDHA,ANISESCENT GOLDENROD,INDIAN GINSENG
720,MEDICAL MARIJUANA,HEMP EXTRACT,"[13, 5, 4, 9, 3, 1, 12, 27, 13, 1, 18, 9, 10, ...","[8, 5, 13, 16, 27, 5, 24, 20, 18, 1, 3, 20, 0,...",1,0,MELIA AZEDARACH,MARIJUANA,HEMP EXTRACT,MEUM ATHAMANTICUM,MENTHA SPICATA
1936,CRANBERRY GELTABS,CRANBERRY,"[3, 18, 1, 14, 2, 5, 18, 18, 25, 27, 7, 5, 12,...","[3, 18, 1, 14, 2, 5, 18, 18, 25, 0, 0, 0, 0, 0...",1,0,CRANBERRY,VACCINIUM MACROCARPON,VACCINIUM PALLIDUM,GERANIUM MACULATUM,CANDLEBERRY
373,NATURAL ALTERNA FLAXSEED OIL PLUS LECITHIN,FLAX SEED,"[14, 1, 20, 21, 18, 1, 12, 27, 1, 12, 20, 5, 1...","[6, 12, 1, 24, 27, 19, 5, 5, 4, 0, 0, 0, 0, 0,...",1,0,FRAGRANT EVERLASTING,FENUGREEK,CABLIN PATCHOULI,CAULOPHYLLUM THALICTROIDES,PSEUDOGNAPHALIUM OBTUSIFOLIUM
2460,FENUGREEK HERBAL SUPPLEMENT,TRIGONELFA FOENUM,"[6, 5, 14, 21, 7, 18, 5, 5, 11, 27, 8, 5, 18, ...","[20, 18, 9, 7, 15, 14, 5, 12, 6, 1, 27, 6, 15,...",1,0,FENUGREEK,TRIGONELFA FOENUM,TRIGONELLA FOENUM,FEVERFEW,TANACETUM PARTHENIUM


In [29]:
subsample = dUnique_df.sample(20)
subsample.rename(columns={"dUnique_seq_padded": 'Processed_dUnique_label'}, inplace=True)

In [30]:
sanity = find_Model_ranks(model, df=subsample, targets= dUnique_df, equivalence_set=equivalence_set, ranks=5, find_related_rank=False, report_distances=False)

In [53]:
sanity.loc[:, sanity.columns != "Processed_dUnique_label"]

,dUnique_label,rank1,rank2,rank3,rank4,rank5
947,IPOMOEA DUMOSA,IPOMOEA DUMOSA,IPOMOEA PANDURATA,IPOMOEA WOLCOTTIANA,ERIOGONUM ALATUM,SPOONLEAF YUCCA
751,GAVILANA,GAVILANA,URENA LOBATA,NEUROLAENA LOBATA,VICIA FABA,GUARANA
416,CLINOPODIUM NEPETA,CLINOPODIUM NEPETA,ASPHODELINE LUTEA,CLINOPODIUM POLYCEPHALUM,CLINOPODIUM BOLIVIANUM,LIRIODENDRON TULIPIFERA
899,HU LU BA,HU LU BA,TRIGONELLA FOENUM,GREEK HAY,HONEY BUCHU,TRIGONELFA FOENUM
1859,YERBA MATE,YERBA MATE,PARAGUAY TEA,ILEX PARAGUARIENSIS,PARAGUAYAN TEA,BRAZILIAN TEA
823,GRINDELIA CILIATA,GRINDELIA CILIATA,TRIGONELLA CORNICULATA,ANGELICA PINNATA,GARDENIA THUNBERGIA,RADICCHIO
1047,LIQUIRITIA OFFICINARUM WHOLE,LIQUIRITIA OFFICINARUM WHOLE,LIQUIRITIA OFFICINALIS WHOLE,CULTIVATED LICORICE WHOLE,URAL LICORICE WHOLE,GUANG GUO GAN CAO WHOLE
1629,SQUAWMINT TICKWEED,SQUAWMINT TICKWEED,STINKINGWEED,MARSHMILKWEED,MENTHA PULEGIUM,EUPATORIUM PURPUREUM
1702,TEUCRIUM BOTRYS,TEUCRIUM BOTRYS,TEUCRIUM CHAMAEDRYS,COMMON DUBBLETJIE,FOENICULUM VULGARE,PUNCTUREVINE CALTROP
531,CURCUMA LONGA,CURCUMA LONGA,CURCUMA,COMMON TURMERIC,TURMERIC,YELLOW GINGER
